In [78]:
import requests
import pandas as pd

NYC API Connection

In [106]:
import pandas as pd
from sodapy import Socrata

# Number of Records limit
nyc_limit = 500
la_limit = 500
chi_limit = 500

# Connect to NYC API
NYCAppToken = '3u5hcZ6WwKere5Mb5nm5S9mT2'
nyc_client = Socrata("data.cityofnewyork.us",
                 NYCAppToken,
                 username="Cameron.Suddreth@du.edu",
                 password="COMP4447groupproject")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
nyc_results = nyc_client.get("5uac-w243", limit=nyc_limit)

# Convert to pandas DataFrame
nyc_df = pd.DataFrame.from_records(nyc_results)


LA API Connection

In [107]:

# Example authenticated client (needed for non-public datasets):
LA_AppToken = 'mEU8HkgWCvfkWLHKGxfiUFecc'
la_client = Socrata("data.lacity.org",
                 LA_AppToken,
                 username="Cameron.Suddreth@du.edu",
                 password="COMP4447groupproject")


la_results = la_client.get("2nrs-mtv8", limit=la_limit)

# Convert to pandas DataFrame
la_df = pd.DataFrame.from_records(la_results)

Chicago API Connection

In [108]:
CHI_AppToken = '6rxQVr5BfXAbUUccKTodxYVdj'
chi_client = Socrata("data.cityofchicago.org",
                    CHI_AppToken,
                    username="Cameron.Suddreth@du.edu",
                    password="COMP4447groupproject")
chi_results = chi_client.get("9hwr-2zxp", limit=chi_limit)
chi_df = pd.DataFrame.from_records(chi_results)

In [ ]:
# # results_df.head(10).to_csv('head_output.csv', index = False)
# print(len(nyc_df))
# print(len(la_df))
# print(len(chi_df))
# la_columns = la_df.columns.tolist()
# nyc_columns = nyc_df.columns.tolist()
# chi_columns = chi_df.columns.tolist()
# print(f'LA: {la_columns}')
# print(f'NYC: {nyc_columns}')
# print(f'CHI: {chi_columns}')


In [109]:
import numpy as np

nyc_df['Weapon'] = np.nan
chi_df['Weapon'] = np.nan
chi_df['Time'] = np.nan
chi_df['Victim Sex'] = np.nan
nyc_df['City'] = 'NYC'
la_df['City'] = "LA"
chi_df['City'] = 'CHI'


chi_df.head()

,date,location,district,block,y_coordinate,latitude,description,location_description,updated_on,community_area,...,fbi_code,longitude,beat,primary_type,arrest,id,Weapon,Time,Victim Sex,City
0,2022-12-31T23:55:00.000,"{'latitude': '41.878373984', 'human_address': ...",015,055XX W ADAMS ST,1898786,41.878373984,RECKLESS FIREARM DISCHARGE,RESIDENCE - YARD (FRONT / BACK),2023-11-15T15:41:31.000,25,...,15,-87.762731975,1522,WEAPONS VIOLATION,False,13024128,NaN,NaN,NaN,CHI
1,2022-12-31T23:50:00.000,"{'latitude': '42.009107852', 'human_address': ...",024,014XX W LUNT AVE,1946614,42.009107852,AUTOMOBILE,ALLEY,2023-01-07T15:41:08.000,1,...,07,-87.666843608,2431,MOTOR VEHICLE THEFT,False,12938337,NaN,NaN,NaN,CHI
2,2022-12-31T23:50:00.000,"{'latitude': '41.882256634', 'human_address': ...",015,055XX W WASHINGTON BLVD,1900199,41.882256634,DOMESTIC BATTERY SIMPLE,APARTMENT,2023-01-07T15:41:08.000,25,...,08B,-87.763744015,1523,BATTERY,False,12938426,NaN,NaN,NaN,CHI
3,2022-12-31T23:47:00.000,"{'latitude': '41.72781203', 'human_address': '...",022,091XX S LAFLIN ST,1844124,41.72781203,UNLAWFUL POSSESSION - HANDGUN,APARTMENT,2023-01-07T15:41:08.000,73,...,15,-87.660399127,2221,WEAPONS VIOLATION,True,12938340,NaN,NaN,NaN,CHI
4,2022-12-31T23:47:00.000,"{'latitude': '41.72781203', 'human_address': '...",022,091XX S LAFLIN ST,1844124,41.72781203,SIMPLE,STREET,2023-01-07T15:41:08.000,73,...,08A,-87.660399127,2221,ASSAULT,True,12938351,NaN,NaN,NaN,CHI


In [110]:

la_df = la_df[['dr_no', 'date_rptd', 'date_occ', 'crm_cd', 'crm_cd_desc', 'weapon_desc', 'vict_sex', 'lat', 'lon', 'City']]
nyc_df = nyc_df[['cmplnt_num', 'cmplnt_fr_dt', 'cmplnt_fr_tm','ky_cd', 'ofns_desc', 'Weapon', 'vic_sex', 'latitude','longitude', 'City']]
chi_df = chi_df[['id', 'date','Time', 'iucr','description','Weapon', 'Victim Sex','latitude','longitude', 'City']]
generic_columns = ['Case Number', 'Date', 'Time', 'Crime Code', 'Crime Description','Weapon', 'Victim Sex', 'Latitude', 'Longitude', 'City']

# la_df.head(15)
# nyc_df.head(15)
# chi_df.head(15)

In [111]:
# Rename the columns
la_df.columns = generic_columns
nyc_df.columns = generic_columns
chi_df.columns = generic_columns


In [112]:
combined_df = pd.concat([la_df, nyc_df, chi_df],ignore_index=True)
# combined_df.sample(25)
print(len(combined_df))
combined_df


1500


,Case Number,Date,Time,Crime Code,Crime Description,Weapon,Victim Sex,Latitude,Longitude,City
0,190326475,2020-03-01T00:00:00.000,2020-03-01T00:00:00.000,510,VEHICLE - STOLEN,NaN,M,34.0375,-118.3506,LA
1,200106753,2020-02-09T00:00:00.000,2020-02-08T00:00:00.000,330,BURGLARY FROM VEHICLE,NaN,M,34.0444,-118.2628,LA
2,200320258,2020-11-11T00:00:00.000,2020-11-04T00:00:00.000,480,BIKE - STOLEN,NaN,X,34.021,-118.3002,LA
3,200907217,2023-05-10T00:00:00.000,2020-03-10T00:00:00.000,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),NaN,M,34.1576,-118.4387,LA
4,220614831,2022-08-18T00:00:00.000,2020-08-17T00:00:00.000,354,THEFT OF IDENTITY,NaN,M,34.0944,-118.3277,LA
5,231808869,2023-04-04T00:00:00.000,2020-12-01T00:00:00.000,354,THEFT OF IDENTITY,NaN,M,33.9467,-118.2463,LA
6,230110144,2023-04-04T00:00:00.000,2020-07-03T00:00:00.000,354,THEFT OF IDENTITY,NaN,M,34.0415,-118.262,LA
7,220314085,2022-07-22T00:00:00.000,2020-05-12T00:00:00.000,354,THEFT OF IDENTITY,NaN,F,34.0335,-118.3537,LA
8,231309864,2023-04-28T00:00:00.000,2020-12-09T00:00:00.000,354,THEFT OF IDENTITY,NaN,F,33.9911,-118.2521,LA
9,211904005,2020-12-31T00:00:00.000,2020-12-31T00:00:00.000,624,BATTERY - SIMPLE ASSAULT,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",M,34.2336,-118.4535,LA


In [113]:
combined_df['str_length'] = combined_df['Crime Description'].str.len()
number_of_crimes = combined_df['Crime Description'].value_counts()
print(type(combined_df['Crime Description'].unique().tolist()))

# crime_data = combined_df[['City', 'Crime Description']]
# city_crime_counts = crime_data.groupby(['City', 'Crime Description']).size().reset_index(name='Count')
# most_frequent_crime = city_crime_counts.groupby('City').apply(lambda x: x.loc[x['Count'].idxmax()]).reset_index(drop=True)
# city_crime_counts.to_excel("city.xlsx")  
# print("City with the Most Frequent Crime:")
# print(most_frequent_crime[['City', 'Crime Description', 'Count']])

<class 'list'>


[theft, assault, murder, rape]

In [ ]:
# !pip install transformers
# !pip3 install torch torchvision

In [104]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
crimes = combined_df['Crime Description'].unique().tolist()

encoded_crimes = [tokenizer.encode_plus(crime, add_special_tokens=True, return_tensors='pt') for crime in crimes]

print(encoded_crimes)
with torch.no_grad():
    embeddings = [model(**crime)['pooler_output'] for crime in encoded_crimes]


similarity_scores = []

for i in range(len(embeddings)):
    for j in range(i+1, len(embeddings)):
        similarity_score = cosine_similarity(embeddings[i], embeddings[j])
        pair_of_words = (i, j)
        similarity_scores.append((similarity_score, pair_of_words))

# print(f"Similarity Scores: {similarity_scores}")



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'input_ids': tensor([[ 101, 4316, 1011, 7376,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}, {'input_ids': tensor([[  101, 20934, 10623, 28221,  2013,  4316,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[ 101, 7997, 1011, 7376,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}, {'input_ids': tensor([[  101,  4497, 26644,  1011,  2882, 11933,  1006,  1002, 20317,  1012,
          5890,  1004,  2058,  1007,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, {'input_ids': tensor([[  101, 11933,  1997,  4767,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}, {'input_ids': tensor([[ 101, 6046, 1011, 3722, 6101,  102]]), 'token_type_ids': tens

In [102]:
print(type(similarity_scores))
scores = []
counter = 0
# for arr in similarity_scores:
#     if counter <100:
#         print(arr)
#     counter +=1

for arr in similarity_scores:
    score = arr[0][0]
    scores.append(score)

print(scores)


<class 'list'>
[array([0.9468013], dtype=float32), array([0.9937383], dtype=float32), array([0.9510783], dtype=float32), array([0.98331577], dtype=float32), array([0.9689156], dtype=float32), array([0.9879184], dtype=float32), array([0.952525], dtype=float32), array([0.9694822], dtype=float32), array([0.97785115], dtype=float32), array([0.97846556], dtype=float32), array([0.96670383], dtype=float32), array([0.99049664], dtype=float32), array([0.99198616], dtype=float32), array([0.9782689], dtype=float32), array([0.98880434], dtype=float32), array([0.9448463], dtype=float32), array([0.9748577], dtype=float32), array([0.7392649], dtype=float32), array([0.9857836], dtype=float32), array([0.9878335], dtype=float32), array([0.98551685], dtype=float32), array([0.75036144], dtype=float32), array([0.97916704], dtype=float32), array([0.99107146], dtype=float32), array([0.95776325], dtype=float32), array([0.95041937], dtype=float32), array([0.91896945], dtype=float32), array([0.9870237], dtype=f

In [114]:
import folium
 
df = combined_df
 
 
# Convert Latitude and Longitude to float
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
 
# Create a map centered around the mean of latitude and longitude
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=10)
 
# Add markers for each crime location
for index, row in df.iterrows():
    # Check for NaN values
    if not pd.isnull(row['Latitude']) and not pd.isnull(row['Longitude']):
        folium.Marker(location=[row['Latitude'], row['Longitude']], popup=row['Case Number']).add_to(m)
 
# Save the map
filename = 'crime_map.html'  # Specify a full file path
m.save(filename)